In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Click <a href="javascript:code_toggle()">here</a> to see the code.''')

In [2]:
import ipywidgets as ipw
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('seaborn-poster')

from tabulate import tabulate

def matProps(mat):
    if mat == 'CuNi10':
        E = 140e9
        nu = 0.35
        rho = 8940
    elif mat == 'Cu':
        E = 110e9
        nu = 0.35
        rho = 7940
    elif mat == 'Al':
        E = 69e9
        nu = 0.35
        rho = 2700
    return E, nu, rho

def tubeFinParams(config):
    if config == 'EK2334':
        dx = 23e-3
        dy = 34e-3
        R = (15e-3)/2
        tFin = 0.18e-3
        tTube = 0.80e-3
    if config == 'EK2334 (VaCN)':
        dx = 23e-3
        dy = 34e-3
        R = (15e-3)/2
        tFin = 0.14e-3
        tTube = 0.80e-3
    if config == 'CK1619':
        dx = 16e-3
        dy = 19e-3
        R = (10.55e-3)/2
        tFin = 0.12e-3
        tTube = 0.60e-3
    if config == 'UCK1212':
        dx = 12e-3
        dy = 12e-3
        R = (7.5e-3)/2
        tFin = 0.12e-3
        tTube = 0.50e-3
    return dx, dy, R, tFin, tTube

def paras(tubeMat, finMat, config):
    dx, dy, R, tFin, tTube = tubeFinParams(config)
    ETube, nuTube, rhoTube = matProps(tubeMat)
    EFin, nuFin, rhoFin = matProps(finMat)
    return dx, dy, nuTube, R, ETube, rhoFin, rhoTube, tFin, tTube

def getFreqs(ETube, nuTube, dx, dy, nRow, nCol, R, rhoTube, rhoFin, tTube, tFin, L, Nf):
    GTube = ETube/(2*(1 + nuTube))
    betaVals = [4.73004, 7.8532, 10.9956, 14.1372]
    x1 = dx/2 - dy**2/(8*dx)
    x2 = dx/2 + dy**2/(8*dx)
    Rsq = 1/48*nCol*nRow*(4*dx**2*(-1 + nCol**2) +dy**2*(-1 + 4*nRow**2))
    aUnit = dy*(x1 + x2) - np.pi*R**2
    JUnit = (3*dy**3*x1 + 4*dy*x1**3 + dy**3*x2 + 4*dy*x1**2*x2 + 4*dy*x1*x2**2 + 4*dy*x2**3)/24 - np.pi*R**4/2
    aTube = np.pi*(R**2 - (R - tTube)**2)
    ITube = np.pi/4*(R**4 - (R - tTube)**4)
    JTube = np.pi/2*(R**4 - (R - tTube)**4)
    Luc = L/Nf
    NTube = nRow*nCol
    JTotal = JUnit*NTube + aUnit * Rsq
    [A, B, C] = getConstants(1, L)
    fT1 = (1/(2*np.pi))*np.sqrt((ETube*ITube*Rsq*B + NTube*GTube*JTube*C)/(A*(rhoTube*aTube*Rsq + JTotal*rhoFin*tFin/Luc)))
    [A, B, C] = getConstants(2, L)
    fT2 = (1/(2*np.pi))*np.sqrt((ETube*ITube*Rsq*B + NTube*GTube*JTube*C)/(A*(rhoTube*aTube*Rsq + JTotal*rhoFin*tFin/Luc)))
    [A, B, C] = getConstants(3, L)
    fT3 = (1/(2*np.pi))*np.sqrt((ETube*ITube*Rsq*B + NTube*GTube*JTube*C)/(A*(rhoTube*aTube*Rsq + JTotal*rhoFin*tFin/Luc)))
    linearDens = aTube*rhoTube + Nf*aUnit*tFin*rhoFin/L
    fBend = np.array(betaVals)**2*np.sqrt(ETube*ITube/(linearDens*L**4))/(2*np.pi)
    fTwist = np.array([fT1, fT2, fT3])
    return fBend, fTwist

def getConstants(i, L):
        ABCs = [[0.396478*L,198.462/L**3, 4.87772/L], [7.92282e28*L,3.01347e32/L**3, 3.64847e30/L], [0.50586*L,7394.47/L**3, 50.032/L]]
        return ABCs[i-1]

***
### Bending and Twisting Mode Frequencies
<b><font color='#1f77b4'>Bending Mode</font> 

<font color='#ff7f0e'> Twisting Mode</font></b>

In [3]:
def plotter(nRow, nCol, Nf, L, maxFreq, tubeMat, finMat, config):
    dx, dy, nuTube, R, ETube, rhoFin, rhoTube, tFin, tTube = paras(tubeMat, finMat, config)
    fBend, fTwist = getFreqs(ETube, nuTube, dx, dy, nRow, nCol, R, rhoTube, rhoFin, tTube, tFin, L, Nf)
    f = np.append(fBend, fTwist)
    idx = np.argsort(f)
    bt = np.array(['C0']*len(fBend) + ['C1']*len(fTwist))
    bt = bt[idx]
    f = f[idx]
    plt.bar(range(len(f)), f, color = bt)
    plt.ylabel('Freq (Hz)')
    plt.ylim([0, 1000])
    plt.xticks([])
    plt.plot(plt.xlim(), [maxFreq, maxFreq], color = 'C3' )
    plt.show()
    freqRounded = ['Freq (Hz): '] + list(int(round(num)) for num in f)
    modeType = np.array(['Bend']*len(fBend) + ['Twist']*len(fTwist))
    modeType = ['ModeType: '] + list(modeType[idx])
    print(tabulate([freqRounded, modeType]))
    
interactive_plot = ipw.interactive(plotter,
                               nRow = (2, 12, 1),
                               nCol=(2, 12, 2),
                               Nf = (107, 307, 10),
                               L = (0.02, 2*0.48, 0.02),
                               maxFreq = (0, 300, 10),
                               tubeMat = ['CuNi10', 'Cu', 'Al'],
                               finMat = ['Cu', 'CuNi10', 'Al'],
                               config = ['EK2334', 'EK2334 (VaCN)', 'CK1619', 'UCK1212']
                               )
display(interactive_plot)

interactive(children=(IntSlider(value=7, description='nRow', max=12, min=2), IntSlider(value=6, description='n…

### Effect of Fin Thickness on  First Bending Mode and First Twisting Mode

In [4]:
def thickEffect(nRow, nCol, Nf, L, maxFreq, tubeMat, finMat, config):
    dx, dy, nuTube, R, ETube, rhoFin, rhoTube, tFin, tTube = paras(tubeMat, finMat, config)
    tFins = np.linspace(0.1, 1.5, 41)*tFin
    fB1 = 0*tFins
    fT1 = 0*tFins
    for i in range(len(tFins)):
        fBend, fTwist = getFreqs(ETube, nuTube, dx, dy, nRow, nCol, R, rhoTube, rhoFin, tTube, tFins[i], L, Nf)
        fB1[i] = fBend[0]
        fT1[i] = fTwist[0]
    plt.plot(tFins/tFin, fB1, color = 'C0', linewidth = 6)
    plt.plot(tFins/tFin, fT1, color = 'C1', linewidth = 6)
    plt.ylabel('Freq (Hz)')

    xlabelStr = '$t_\mathrm{Fin}}$/%1.2f mm' %(tFin*1000)
    plt.xlabel(xlabelStr)
    plt.ylim([0, 1000])
    plt.xticks([0.1, 0.5, 1, 1.5])
    plt.xlim([min(tFins/tFin), max(tFins/tFin)])
    plt.plot(plt.xlim(), [maxFreq, maxFreq], color = 'C3' )
    
        
interactive_plot2 = ipw.interactive(thickEffect,
                               nRow = (2, 12, 1),
                               nCol=(2, 12, 2),
                               Nf = (107, 307, 10),
                               L = (0.02, 2*0.48, 0.02),
                               maxFreq = (0, 300, 10),
                               tubeMat = ['CuNi10', 'Cu', 'Al'],
                               finMat = ['Cu', 'CuNi10', 'Al'],
                               config = ['EK2334', 'EK2334 (VaCN)', 'CK1619', 'UCK1212']
                               )
display(interactive_plot2)

interactive(children=(IntSlider(value=7, description='nRow', max=12, min=2), IntSlider(value=6, description='n…